In [1]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.io import loadmat, savemat
import tensorflow as tf
from tensorflow.keras import models,Model
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv2D, AveragePooling2D, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.optimizers import Adam,RMSprop
from  tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.datasets import mnist

In [2]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [3]:
# def normal_signal(data):
#     mean, var = data.mean(axis=0), data.var(axis=0)
#     data_norm = (data - mean) / np.sqrt(var)
#     data_norm = data_norm.astype("float32")
#
#     return data_norm
#


In [4]:
data_x_ori = loadmat('huidu_real_label.mat')['huidu']
print(data_x_ori.shape)
print(data_x_ori)

(1024, 1000)
[[0.45867347 0.34031991 0.04790749 ... 0.45821539 0.46929334 0.42178952]
 [0.60255102 0.33756205 0.13050661 ... 0.47823157 0.46578614 0.50913306]
 [0.6377551  0.34583563 0.3188326  ... 0.50957002 0.46586076 0.49986092]
 ...
 [0.29540816 0.3877551  0.41905286 ... 0.50909826 0.45914484 0.49958275]
 [0.26938776 0.28461114 0.33975771 ... 0.47398571 0.45071263 0.36383866]
 [0.32806122 0.15057915 0.36343612 ... 0.51219841 0.51637937 0.46713027]]


In [5]:
# data_x_normal = normal_signal(data_x_ori)
# print(data_x_normal)


In [6]:
data_x_swapaxes = data_x_ori.swapaxes(1,0)
print(data_x_swapaxes.shape)
print(data_x_swapaxes)

(1000, 1024)
[[0.45867347 0.60255102 0.6377551  ... 0.29540816 0.26938776 0.32806122]
 [0.34031991 0.33756205 0.34583563 ... 0.3877551  0.28461114 0.15057915]
 [0.04790749 0.13050661 0.3188326  ... 0.41905286 0.33975771 0.36343612]
 ...
 [0.45821539 0.47823157 0.50957002 ... 0.50909826 0.47398571 0.51219841]
 [0.46929334 0.46578614 0.46586076 ... 0.45914484 0.45071263 0.51637937]
 [0.42178952 0.50913306 0.49986092 ... 0.49958275 0.36383866 0.46713027]]


In [7]:
data_x = data_x_swapaxes.reshape((-1,32,32))
print(data_x.shape)
print(data_x)

(1000, 32, 32)
[[[0.45867347 0.60255102 0.6377551  ... 0.47755102 0.55969388 0.51428571]
  [0.33316327 0.16581633 0.17908163 ... 0.50969388 0.5122449  0.55510204]
  [0.61530612 0.58163265 0.4122449  ... 0.54081633 0.57908163 0.6       ]
  ...
  [0.58520408 0.50153061 0.52397959 ... 0.23163265 0.21581633 0.34438776]
  [0.48571429 0.51071429 0.42346939 ... 0.61071429 0.56479592 0.54642857]
  [0.4744898  0.57091837 0.67244898 ... 0.29540816 0.26938776 0.32806122]]

 [[0.34031991 0.33756205 0.34583563 ... 0.61059018 0.5394374  0.50689465]
  [0.46773304 0.52013238 0.5019305  ... 0.45559846 0.59018202 0.60507446]
  [0.58521787 0.51902923 0.49145063 ... 0.2046332  0.07335907 0.02978489]
  ...
  [0.3386652  0.31549917 0.29895201 ... 0.75455047 0.91781577 0.95091009]
  [0.82735797 0.57859901 0.43960287 ... 0.16105902 0.20518478 0.33811362]
  [0.61389961 0.7854385  0.73579702 ... 0.3877551  0.28461114 0.15057915]]

 [[0.04790749 0.13050661 0.3188326  ... 0.47742291 0.59801762 0.6376652 ]
  [0.55

In [8]:
# print(data_x[0,:,:])

In [9]:
# data_x_normal = data_x / 255
# print(data_x_normal)
# print(data_x_normal[0,:,:])


In [10]:
data_x_dims = np.expand_dims(data_x, axis=-1)
print(data_x_dims.shape)
print(data_x_dims)

(1000, 32, 32, 1)
[[[[0.45867347]
   [0.60255102]
   [0.6377551 ]
   ...
   [0.47755102]
   [0.55969388]
   [0.51428571]]

  [[0.33316327]
   [0.16581633]
   [0.17908163]
   ...
   [0.50969388]
   [0.5122449 ]
   [0.55510204]]

  [[0.61530612]
   [0.58163265]
   [0.4122449 ]
   ...
   [0.54081633]
   [0.57908163]
   [0.6       ]]

  ...

  [[0.58520408]
   [0.50153061]
   [0.52397959]
   ...
   [0.23163265]
   [0.21581633]
   [0.34438776]]

  [[0.48571429]
   [0.51071429]
   [0.42346939]
   ...
   [0.61071429]
   [0.56479592]
   [0.54642857]]

  [[0.4744898 ]
   [0.57091837]
   [0.67244898]
   ...
   [0.29540816]
   [0.26938776]
   [0.32806122]]]


 [[[0.34031991]
   [0.33756205]
   [0.34583563]
   ...
   [0.61059018]
   [0.5394374 ]
   [0.50689465]]

  [[0.46773304]
   [0.52013238]
   [0.5019305 ]
   ...
   [0.45559846]
   [0.59018202]
   [0.60507446]]

  [[0.58521787]
   [0.51902923]
   [0.49145063]
   ...
   [0.2046332 ]
   [0.07335907]
   [0.02978489]]

  ...

  [[0.3386652 ]
   [0

In [11]:
# data_x_normal = tf.cast(data_x, dtype = tf.float32) / 255
# print(data_x_normal)
# print(data_x_normal[0,:,:])

In [12]:
# data_x_dims = tf.expand_dims(data_x_normal, axis=-1)
#

In [13]:
data_y_ori = loadmat('huidu_real_label.mat')['real_label']
print(data_y_ori.shape)
print(data_y_ori)

(1, 1000)
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
  3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
  3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
  3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
  4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
  4 4 4 4 4 4 4 4 4 4 4 4 4 

In [14]:
data_y_dims = np.squeeze(data_y_ori, axis=0)
print(data_y_dims.shape)
print(data_y_dims)
# data_y_dims = np.expand_dims(data_y_dims, axis=-1)
# print(data_y_dims)

(1000,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 

In [15]:
data_y_onehot = np.eye(10)[data_y_dims]
print(data_y_onehot.shape)
print(data_y_onehot)

(1000, 10)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [16]:
# data_y = tf.cast(data_y_ori, dtype=tf.float32)
# print(data_y)

In [17]:
# data_y_dims = tf.squeeze(data_y, axis=0)
# print(data_y_dims)
# data_y_dims = tf.expand_dims(data_y_dims, axis=-1)
# print(data_y_dims)

In [18]:
train_x, test_x, train_y, test_y = train_test_split(data_x_dims, data_y_onehot, test_size=0.3)
# print(train_x.shape, train_x)
# print(train_y.shape, train_y)
# print(test_x.shape, test_x)
# print(test_y.shape, test_y)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(700, 32, 32, 1)
(700, 10)
(300, 32, 32, 1)
(300, 10)


In [19]:
# dataset = tf.data.Dataset.from_tensor_slices((data_x_dims, data_y_dims)).shuffle(data_x.shape[0]).batch(16)

In [20]:
# def cnn():
#     inputs = Input(shape=(32, 32, 1))
#     cnn1 = Conv2D(filters=64, kernel_size=(5, 5), strides=(1, 1), padding='valid', activation='tanh')(inputs)  # 6, 24*24
#     max1 = MaxPooling2D(pool_size=(2, 2))(cnn1)  # 6, 12*12
#     cnn2 = Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='tanh')(max1)  # 12, 8*8
#     max2 = MaxPooling2D(pool_size=(2, 2))(cnn2)  # 12, 4*4
#
#     outputs = Dense(units=6, activation='softmax')(max2)  # 10
#
#     model = Model(inputs, outputs)
#     return model

In [21]:
# model = cnn()
# model.summary()
# model.compile(optimizer=Adam(learning_rate=0.1), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#

In [22]:
# history = model.fit(train_x, train_y, epochs=100, batch_size=64, shuffle='True', validation_data=(test_x, test_y))

In [23]:
model = models.Sequential()
model.add(layers.Conv2D(4,(10,10),padding='same',activation='relu',input_shape=(32,32,1)))
model.add(layers.MaxPooling2D((3,3),strides=2))
model.add(layers.Conv2D(4,(5,5),padding='same',activation='relu'))
model.add(layers.MaxPooling2D((3,3),strides=2))
model.add(layers.Conv2D(8,(3,3),padding='same',activation='relu'))
model.add(layers.Conv2D(8,(3,3),padding='same',activation='relu'))
model.add(layers.Conv2D(8,(3,3),padding='same',activation='relu'))
model.add(layers.MaxPooling2D((3,3),strides=2))
model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add((layers.Dense(10,activation='softmax')))






In [24]:
model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [25]:
model.summary()

save = ModelCheckpoint(
                                filepath='./model/huidu_real_label.h5',
                                monitor='val_accuracy',
                                save_best_only=True,

                                mode='max',
                        )

lr = ReduceLROnPlateau(monitor='val_accuracy',
                       factor=0.5,
                       patience=30,
                       min_delta=0.0001,
                       min_lr=0

                        )

callback_list = [save,lr]




history = model.fit(train_x,train_y,epochs=250, batch_size=16,validation_data=(test_x, test_y),callbacks=callback_list)
#绘制训练过程中的损失曲线和精度曲线



#---------------------------histpry-----------------------------
history.history
savemat('./model/huidu_real_label-loss+acc.mat', dict(
    loss=history.history['loss'], acc=history.history['accuracy'],
    val_loss=history.history['val_loss'], val_acc=history.history['val_accuracy']
                                                )
        )



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 4)         404       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 4)         404       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 4)           0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 8)           296       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 8)           584       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 8)           5

In [26]:
#----------------------测试---------------------------

In [26]:
new_model1 = models.load_model('E:/learn/code/dx7/model/huidu_real_label.h5')
# new_model1.summary()

In [27]:
print(new_model1.evaluate(test_x, test_y))

10/10 [==============================] - 0s 3ms/step - loss: 0.7036 - accuracy: 0.9433
[0.7035937905311584, 0.9433333277702332]


In [34]:
savemat('./model/huidu_real_label-Intput.mat', dict(label=test_y))



In [30]:
output = new_model1.predict(test_x)
output


array([[1.94854346e-18, 2.19904560e-18, 3.15410421e-15, ...,
        3.20979520e-21, 0.00000000e+00, 3.98233682e-33],
       [9.57538917e-17, 4.01444516e-32, 3.75643988e-10, ...,
        3.88451597e-15, 8.28427660e-33, 1.00000000e+00],
       [2.32154524e-20, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 1.08851800e-31, 1.53056662e-15, ...,
        9.89751693e-22, 7.19195015e-09, 0.00000000e+00],
       [9.60396555e-16, 2.35759399e-18, 5.18826791e-16, ...,
        1.26742475e-17, 0.00000000e+00, 1.45499324e-35],
       [0.00000000e+00, 0.00000000e+00, 4.75705945e-18, ...,
        1.15473073e-23, 7.61323236e-08, 0.00000000e+00]], dtype=float32)

In [31]:
savemat('./model/huidu_real_label-Output.mat', dict(outputs=output))


In [30]:
# import matplotlib.pyplot as plt
#
# acc = history.history['accuracy']
# loss = history.history['loss']
# epochs = range(1,len(acc) + 1)
# plt.plot(epochs,acc,'bo',label='Training acc')
# plt.title('Training  accuracy')
# plt.legend()
# plt.figure()
# plt.plot(epochs,loss,'bo',label='Training loss')
# plt.title('Training  loss')
# plt.legend()
# plt.show()
#
# print(history.history.keys())
# print(model.evaluate(test_x, test_y))

In [31]:
# ----------------------------最佳模型的参数导出-------------------------------

In [32]:
# model.save('model_v3_lr=0.0001_epoch1.h5', save_traces=False)
new_model2 = models.load_model('E:/learn/code/dx7/model/huidu_real_label.h5')
# new_model2.summary()
# print(outputs_test.shape)
# print(outputs_test)
# savemat('outputs_tset_v3_lr=0.0001_epoch1.mat', dict(outputs=outputs_test))


In [33]:
weights = np.array(new_model2.get_weights(), dtype=object)
# print(weights.shape)
# # (4,)权重由4部分组成
# # print(weights[0].shape)
# # (784, 64)dense_1 w1
# # print(weights[1].shape)
# # (64,)dense_1 b1
# # print(weights[2].shape)
# # (64, 10)dense_2 w2
# # print(weights[3].shape)
# # (10,)dense_2 b2
# # print(weights[4].shape)
# # print(weights[5].shape)
savemat('./model/huidu_real_label-weight.mat', dict(weights=weights))
# # np.save('weights.npy', weights)

[[[ 0.27858657 -0.11032599  0.24256352  0.36759272]
  [ 0.08975185 -0.10723591 -0.02123244  0.00288844]
  [ 0.05501748 -0.04072938  0.2315243   0.1318572 ]
  [ 0.20920897  0.18813749  0.12406431  0.25484413]
  [ 0.25686973  0.05315118 -0.08031166  0.24063075]]

 [[ 0.19313557 -0.08968989  0.1354165   0.21789181]
  [ 0.23441102 -0.02861835  0.07250351  0.11338403]
  [-0.02714243  0.13866045  0.22989765  0.234159  ]
  [ 0.25558934  0.09735649  0.00288611  0.3179827 ]
  [-0.01781337  0.22774687  0.21401136  0.3348234 ]]

 [[ 0.11895558  0.07083923 -0.0089698   0.3306979 ]
  [ 0.21509655  0.22910346 -0.1304356   0.22664465]
  [ 0.14243919 -0.05880789 -0.0623595   0.12688187]
  [ 0.13814406  0.05518676  0.05876957  0.3520218 ]
  [ 0.32759848 -0.00542256  0.07507815  0.03140919]]

 [[ 0.21444057  0.02549149  0.18373482  0.00173025]
  [ 0.22021447  0.10022712  0.07573871  0.08809531]
  [ 0.11397597  0.23758484 -0.15114763 -0.01247049]
  [ 0.02482653  0.10542525  0.12698376  0.12001035]
  [ 0.

In [29]:

# ----------------------------------2.获取每一层输出------------------------

(1800, 32, 32, 1)
[[[[ 5.19487179e-01]
   [ 6.79487179e-01]
   [ 8.84615385e-01]
   ...
   [ 2.62051282e-01]
   [ 3.20000000e-01]
   [ 3.66666667e-01]]

  [[ 3.13333333e-01]
   [ 2.17948718e-01]
   [ 2.23076923e-01]
   ...
   [ 6.75384615e-01]
   [ 5.11794872e-01]
   [ 3.51282051e-01]]

  [[ 3.63589744e-01]
   [ 4.02051282e-01]
   [ 3.97435897e-01]
   ...
   [ 3.02051282e-01]
   [ 4.71794872e-01]
   [ 4.93846154e-01]]

  ...

  [[ 4.76410256e-01]
   [ 4.38974359e-01]
   [ 4.48205128e-01]
   ...
   [ 5.68717949e-01]
   [ 4.22564103e-01]
   [ 2.70256410e-01]]

  [[ 3.15384615e-01]
   [ 4.18461538e-01]
   [ 4.88205128e-01]
   ...
   [ 5.20000000e-01]
   [ 6.34358974e-01]
   [ 6.24102564e-01]]

  [[ 4.71282051e-01]
   [ 3.73846154e-01]
   [ 4.23076923e-01]
   ...
   [ 3.68205128e-01]
   [ 3.11282051e-01]
   [ 2.44102564e-01]]]


 [[[ 3.78680927e-04]
   [-7.41247772e-04]
   [-1.24642424e-03]
   ...
   [-1.53083779e-04]
   [-4.79393939e-04]
   [ 2.45739750e-04]]

  [[ 1.26334403e-03]
   [ 1.